In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import kinodyn_um.urdfparser as u2c
import kinodyn_um.utils.plucker as pluck
import numpy as np
import casadi as ca
import matplotlib.pyplot as plt

In [2]:
alpha = u2c.URDFparser()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf',
    'alpha_5_robot.urdf'
)
alpha.from_file(path_to_urdf)

root = "base_link"
tip = "alpha_standard_jaws_base_link"

joint_min = np.array([1.00, 0.01, 0.01, 0.01])
joint_max = np.array([5.50, 3.40, 3.40, 5.70])

n_joints = alpha.get_n_joints(root, tip)
i_X_0fs = alpha.forward_kinematics(root, tip, floating_base = False)

q = ca.SX.sym("q", n_joints)

i_X_0s = i_X_0fs(q)

H4 , R4, p4 = pluck.spatial_to_homogeneous(i_X_0s[4])
FK_eval = ca.vertcat(p4, pluck.rotation_matrix_to_euler(R4, order='xyz'))

In [ ]:
p_des = ca.SX.sym('p_des', 3, 1)
# residual: FK(q) - p_des
res = ca.vertcat(FK_eval[0], FK_eval[1], FK_eval[2], FK_eval[5]) - p_des

# create rootfinder
ik_solver = ca.rootfinder('IK', 'newton', ca.Function('res', [q, p_des], [res]))

# # solve for a target, given an initial guess
# target = [1.0, 1.0, 1.0, 1.0]
# q0     = [0.5, 0.5]
# q_sol  = ik_solver(q0, target)
# print("IK solution:", q_sol)

RuntimeError: .../casadi/core/function_internal.cpp:146: Error calling Newton::init for 'IK':
.../casadi/core/rootfinder.cpp:218: Assertion "n_ == oracle_.nnz_in(iin_)" failed:
Dimension mismatch. Input size is 4, while output size is 3